### Imports, deck creation and definitions

In [4]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata as rd
import random
from collections import OrderedDict
from itertools import combinations
from functools import reduce
import time
from tqdm import tqdm

In [5]:
def choose(n, k):
    """
    A fast way to calculate binomial coefficients by Andrew Dalke (contrib).
    """
    if 0 <= k <= n:
        ntok = 1
        ktok = 1
        for t in range(1, min(k, n-k) + 1):
            ntok *= n
            ktok *= t
            n -= 1
        return ntok // ktok
    else:
        return 0

def get_hand_rank(hand):
    """
    Fast hand evaluator courtesy of 
    https://suffe.cool/poker/evaluator.html
    
    Input hand 'h' with cards in binary representation, i.e.,
    for h = ('Ac', 'Kd', 'Kh', 'Tc', '4d'), get binary representation
    using get_bin_rep function defined below. Here, binary representation is
    bh = (268471337, 134236965, 134228773, 16812055, 279045)
    
    >> get_hand_rank(bh)
    3570
    
    3570 is the unique rank for pair of Kings
    
    """
    q = reduce(lambda x,y: x|y, hand) >> 16
    if (reduce(lambda x,y: x&y, hand) & 0xF000) != 0:
        return flushes[q]
    elif q in uniques:
        return uniques[q]
    else:
        primes = [(c & 0xFF) for c in hand]
        r = reduce(lambda x,y: x*y, primes)
        return balance[r]

def get_bin_rep(card):
    r, s = card[0], card[1]
    return (ranks[r][0] << 16) | (suits[s][0] << 12) | (ranks[r][1] << 8) | ranks[r][2]

# Create deck
suits = {'c':[0b1000,'\u2663'],
         'd':[0b0100,'\u2662'],
         'h':[0b0010,'\u2661'],
         's':[0b0001,'\u2660']}

ranks = {'A': [0b0001000000000000, 12, 41],
         'K': [0b0000100000000000, 11, 37],
         'Q': [0b0000010000000000, 10, 31],
         'J': [0b0000001000000000,  9, 29],
         'T': [0b0000000100000000,  8, 23],
         '9': [0b0000000010000000,  7, 19],
         '8': [0b0000000001000000,  6, 17],
         '7': [0b0000000000100000,  5, 13],
         '6': [0b0000000000010000,  4, 11],
         '5': [0b0000000000001000,  3,  7],
         '4': [0b0000000000000100,  2,  5],
         '3': [0b0000000000000010,  1,  3],
         '2': [0b0000000000000001,  0,  2]}

deck = []
for i in ranks:
    for j in suits:
        deck.append(i+j)

bin_deck = [get_bin_rep(x) for x in deck]
bin_deck_dict = dict(zip(bin_deck, deck))

def get_pretty_suits(bin_card):
    card = bin_deck_dict[bin_card]
    return card[0]+suits[card[1]][1]

In [6]:
"""
Get hand ranks; apply primes ("p") and "qs" ("q") from ranks dict
per Cactus Kevin hash scheme
"""
df = pd.read_csv('poker hand equivalence classes.csv')
df['p'] = 1
df['q'] = 0
for i in range(5):
    df['p'] *= df['c'+str(i+1)].apply(lambda x: ranks[x][2])
    df['q'] |= df['c'+str(i+1)].apply(lambda x: ranks[x][0])

# Create ordered lookup tables for flushes, uniques, and balance of hands    
flush_mask = df.Hand.isin(['RF', 'SF', 'F'])
uniques_mask = df.Hand.isin(['S', 'HC'])

df_flushes = df.loc[flush_mask]
df_uniques = df.loc[uniques_mask]
df_balance = df.loc[~flush_mask & ~uniques_mask]

flush_dict = dict(zip(df_flushes.q, df_flushes.Rank))
flushes = dict(OrderedDict(sorted(flush_dict.items())))

uniques_dict = dict(zip(df_uniques.q, df_uniques.Rank))
uniques = dict(OrderedDict(sorted(uniques_dict.items())))

balance_dict = dict(zip(df_balance.p, df_balance.Rank))
balance = dict(OrderedDict(sorted(balance_dict.items())))

### Choose number of players, deal hole cards, and compute pre-flop equity

In [10]:
%%time
# Deal hole cards
n_players = 6
n_sims = 10**4
hole_cards = random.sample(bin_deck, k=n_players*2)
avail_for_flop = [x for x in bin_deck if x not in hole_cards]

holes1 = hole_cards[:n_players]
holes2 = hole_cards[n_players:]
holes_by_player = [tuple(ele) for ele in list(zip(holes1, holes2))]

# Pretty suits
hole_cards_pretty = list(map(get_pretty_suits, hole_cards))
holes1p = hole_cards_pretty[:n_players]
holes2p = hole_cards_pretty[n_players:]
holes_by_player_pretty = [list(ele) for ele in list(zip(holes1p, holes2p))]

# Compute and print pre-flop equity using Monte Carlo sim
rank_array = np.zeros([n_sims, n_players])
t0 = time.time()
for k in tqdm(range(n_sims)):
    community_cards = tuple(random.sample(avail_for_flop, k=5))

    for i in range(n_players):
        hole = holes_by_player[i]        
        rank_list_iter = []
        
        for hand in combinations(holes_by_player[i]+community_cards, 5):
            rank_list_iter.append(get_hand_rank(hand))

        rank_array[k, i] = min(rank_list_iter)
        
t1 = time.time()
print("{:,.0f}".format(n_sims*n_players/(t1-t0)), "hands per second\n")

wins_array = rd(rank_array, axis=1, method='min')==1
wins_total = (rd(rank_array, axis=1, method='min')==1).sum(axis=1).reshape(-1,1)
preflop_equity = np.sum(wins_array/wins_total, axis=0)/k

print("Pre-flop equity:")
for j in range(n_players):
    print("{:5.0%}".format(preflop_equity[j]), "",
          *holes_by_player_pretty[j])

100%|██████████| 10000/10000 [00:06<00:00, 1661.48it/s]


9,944 hands per second

Pre-flop equity:
  24%  5♠ A♣
  16%  K♠ 5♣
   5%  2♣ 4♡
  41%  J♢ Q♢
   3%  2♢ 4♠
  11%  4♣ 6♠
Wall time: 7.8 s


### Post-flop

In [27]:
# Will be exact equity as # of combos small enough to not have to use Monte Carlo
combos_post_flop = choose(52-2*n_players-3, 2)
rank_array = np.zeros([combos_post_flop, n_players])

flop = tuple(random.sample(avail_for_flop, k=3))
pretty_flop = list(map(get_pretty_suits, flop))
avail_for_turn = [x for x in avail_for_flop if x not in flop]

print("Flop:", *pretty_flop, "\n")

for k, turn_and_river in enumerate(combinations(avail_for_turn, 2)):
    for i in range(n_players):
        rank_list_iter = []
        
        for hand in combinations(holes_by_player[i]+flop+turn_and_river, 5):
            rank_list_iter.append(get_hand_rank(hand))
        
        rank_array[k, i] = min(rank_list_iter)

wins_array = rd(rank_array, axis=1, method='min')==1
wins_total = (rd(rank_array, axis=1, method='min')==1).sum(axis=1).reshape(-1,1)
postflop_equity = np.sum(wins_array/wins_total, axis=0)/combos_post_flop
        
print("Post-flop equity and improvement vs. pre-flop:")
for j in range(n_players):
    print("{:5.0%}".format(postflop_equity[j]), 
          "{:+5.0%}".format(postflop_equity[j]-preflop_equity[j]), "",
          *holes_by_player_pretty[j])

Flop: 7♢ A♣ 5♣ 

Post-flop equity and improvement vs. pre-flop:
  14%   -8%  9♢ 3♢
  10%  -11%  T♠ 6♠
  10%   -4%  5♠ J♣
  18%   -0%  Q♣ 5♢
  24%   +8%  7♣ 8♡
  23%  +15%  6♡ 4♠


### Post-turn, pre-river

In [34]:
# Will be exact equity as # of combos small enough to not have to use Monte Carlo
combos_post_turn = 52-2*n_players-3-1
rank_array = np.zeros([combos_post_turn, n_players])

turn = tuple(random.sample(avail_for_turn, k=1))
pretty_turn = list(map(get_pretty_suits, turn))
avail_for_river = [x for x in avail_for_turn if x not in turn]

print("Flop:", *pretty_flop)
print("Turn:", *pretty_turn, "\n")

for k, river in enumerate(combinations(avail_for_river, 1)):
    for i in range(n_players):
        rank_list_iter = []
        
        for hand in combinations(holes_by_player[i]+flop+turn+river, 5):
            rank_list_iter.append(get_hand_rank(hand))
        
        rank_array[k, i] = min(rank_list_iter)

wins_array = rd(rank_array, axis=1, method='min')==1
wins_total = (rd(rank_array, axis=1, method='min')==1).sum(axis=1).reshape(-1,1)
postturn_equity = np.sum(wins_array/wins_total, axis=0)/combos_post_turn

print("Post-turn equity and improvement vs. flop:")
for j in range(n_players):
    print("{:5.0%}".format(postturn_equity[j]), 
          "{:+5.0%}".format(postturn_equity[j]-postflop_equity[j]), "",
          *holes_by_player_pretty[j])

Flop: 7♢ A♣ 5♣
Turn: 9♠ 

Post-turn equity and improvement vs. flop:
  44%  +31%  9♢ 3♢
  17%   +7%  T♠ 6♠
   8%   -2%  5♠ J♣
  11%   -7%  Q♣ 5♢
  11%  -13%  7♣ 8♡
   8%  -15%  6♡ 4♠


### Post-river final hand rank

In [45]:
rank_array = np.zeros([1, n_players])

river = tuple(random.sample(avail_for_river, k=1))

print("Flop:", *pretty_flop)
print("Turn:", *pretty_turn)
print("River:", *list(map(get_pretty_suits, river)), "\n")

for i in range(n_players):
    rank_list_iter = []
    
    for hand in combinations(holes_by_player[i]+flop+turn+river, 5):
        rank_list_iter.append(get_hand_rank(hand))
    
    rank_array[0, i] = min(rank_list_iter)
        
print("Final hand rankings and improvement vs. turn:")
for j in range(n_players):
    print("{:5.0%}".format(np.sum(rank_array.argmin(axis=1)==j)), 
          "{:+5.0%}".format(np.sum(rank_array.argmin(axis=1)==j)-postturn_equity[j]), "",
          *holes_by_player_pretty[j])

print("\nWinning hand:",
      df.loc[df.Rank==np.min(rank_array), 'HandName'].to_string(index=False))

Flop: 7♢ A♣ 5♣
Turn: 9♠
River: K♠ 

Final hand rankings and improvement vs. turn:
 100%  +56%  9♢ 3♢
   0%  -17%  T♠ 6♠
   0%   -8%  5♠ J♣
   0%  -11%  Q♣ 5♢
   0%  -11%  7♣ 8♡
   0%   -8%  6♡ 4♠

Winning hand:  Pair of Nines
